In [1]:
import glob
import os
import chachifuncs as ccf
import descriptors as d
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import pickle
%matplotlib inline

In [2]:
k_list = [f for f in glob.glob('data/K_descriptors/*.xlsx')]
c_list = [f for f in glob.glob('data/C_descriptors/*.xlsx')]

In [3]:
c_data = pd.DataFrame()
for each in c_list:
    df = pd.read_excel(each)
    c_data = c_data.append(df,ignore_index=True)
#c_data.shape
#remove_n_c = c_data.shape[0]-350

#drop_indices = np.random.choice(c_data.index, remove_n_c, replace=False)
#c_subset = c_data.drop(drop_indices)
#c_subset.shape

In [4]:
k_data = pd.DataFrame()
for each in k_list:
    df = pd.read_excel(each)
    k_data = k_data.append(df,ignore_index=True)
k_data.shape

#remove_n_k = k_data.shape[0]-350

#drop_indices = np.random.choice(k_data.index, remove_n_k, replace=False)
#k_subset = k_data.drop(drop_indices)
#k_subset.shape

(640, 40)

In [5]:
#data = c_subset.append(k_subset)
data = c_data.append(k_data)
data = data.T.drop_duplicates().T

In [6]:
data = data.reset_index(drop = True)

for i in range(len(data)):
    if data.loc[i, ('names')].startswith('CS2_33'):
        data.loc[i, ('label')] = 'LiCoO2'
        data.loc[i, ('lasso')] = 0
    else:
        data.loc[i,('label')] = 'LiFePO4'
        data.loc[i, ('lasso')] = 1

data.label.unique()

array(['LiCoO2', 'LiFePO4'], dtype=object)

In [7]:
print ("Shape of dataset: ",data.shape) #size of dataset
data.head(10) #check 

Shape of dataset:  (1030, 36)


,ch_0,ch_1,ch_2,ch_3,ch_4,ch_5,ch_6,ch_7,ch_8,ch_9,...,dc_8,dc_9,dc_10,dc_11,dc_12,dc_13,dc_14,dc_15,label,lasso
0,216057,-219361,83436.7,-14091.3,3.81096,1.98733,0.0248541,3.90362,3.83951,0.0230099,...,0,0,0,0,0,0,0,0,LiCoO2,0.0
1,81798,-84121.6,32386.1,-5532.19,3.80934,1.74861,3.33851,3.89844,4.11237,0.0254297,...,0,0,0,0,0,0,0,0,LiCoO2,0.0
2,69127.1,-71131,27396.5,-4681.17,3.80416,1.77687,0.519043,3.89746,4.08835,0.0258235,...,0,0,0,0,0,0,0,0,LiCoO2,0.0
3,65078.7,-66829.4,25686.4,-4379.73,3.80788,1.68394,0.325897,3.90346,3.92758,0.0277913,...,0,0,0,0,0,0,0,0,LiCoO2,0.0
4,97590.8,-100242,38551.1,-6579.07,3.81517,1.62222,70.7299,3.90249,3.97789,0.025399,...,0,0,0,0,0,0,0,0,LiCoO2,0.0
5,90553.4,-92961.5,35729.2,-6093.47,3.81388,1.62629,87.808,3.90119,4.00304,0.0259246,...,0,0,0,0,0,0,0,0,LiCoO2,0.0
6,83140.6,-85365.3,32813.8,-5596.76,3.81404,1.60402,0.823952,3.90394,3.94021,0.0275282,...,0,0,0,0,0,0,0,0,LiCoO2,0.0
7,89571.2,-92052.2,35417.5,-6046.67,3.80999,1.6505,57.4664,3.89714,3.87901,0.0252584,...,0,0,0,0,0,0,0,0,LiCoO2,0.0
8,79038.8,-81240.1,31259.6,-5336.77,3.80821,1.66692,7.05149,3.89908,3.84626,0.0284289,...,0,0,0,0,0,0,0,0,LiCoO2,0.0
9,94001.3,-96556.3,37133.1,-6336.83,3.81614,1.62057,7.76953,3.90087,3.84674,0.0249746,...,0,0,0,0,0,0,0,0,LiCoO2,0.0


In [8]:
print("The following features were found: ") #column names 
[each for each in data]

The following features were found: 


['ch_0',
 'ch_1',
 'ch_2',
 'ch_3',
 'ch_4',
 'ch_5',
 'ch_6',
 'ch_7',
 'ch_8',
 'ch_9',
 'ch_10',
 'ch_11',
 'ch_12',
 'ch_13',
 'ch_14',
 'ch_15',
 'ch_16',
 'names',
 'dc_0',
 'dc_1',
 'dc_2',
 'dc_3',
 'dc_4',
 'dc_5',
 'dc_6',
 'dc_7',
 'dc_8',
 'dc_9',
 'dc_10',
 'dc_11',
 'dc_12',
 'dc_13',
 'dc_14',
 'dc_15',
 'label',
 'lasso']

In [9]:
unique = data['label'].unique()
print ("There are", len(unique), "unique batteries in this dataset: ", unique) #how many unique batteries are we looking at 

There are 2 unique batteries in this dataset:  ['LiCoO2' 'LiFePO4']


In [10]:
#split up data into test train set 80/20

train,test = train_test_split(data, test_size=0.2, random_state=1010)
print (train.shape, test.shape)

(824, 36) (206, 36)


In [11]:
#LASSO 
#feature selection
train_lasso_y = train['lasso']
train_lasso_x = train[['ch_4','ch_5','ch_7','ch_8','ch_10','ch_11','dc_4','dc_5','dc_7','dc_10','dc_11']] #all the descriptors
test_lasso_y = test['lasso']
test_lasso_x = test[['ch_4','ch_5','ch_7','ch_8','ch_10','ch_11','dc_4','dc_5','dc_7','dc_10','dc_11']]
#4,5,7,8,10,11

train_lasso_x_scaled  = preprocessing.normalize(train_lasso_x, norm='l1')
test_lasso_x_scaled  = preprocessing.normalize(test_lasso_x, norm='l1')

In [12]:
clf = linear_model.Lasso(alpha=0.004, tol=0.0001 ,max_iter=5000)
#clf.fit(train_lasso_x_scaled,train_lasso_y)
clf.fit(train_lasso_x_scaled,train_lasso_y)
coef = clf.coef_

coef
#4,5,7,8,10,11
#5,7,5

array([-0.        ,  1.80409918, -0.18324826, -0.        , -0.        ,
       -0.        , -0.        ,  1.41705721, -0.        ,  0.        ,
        0.        ])

In [13]:
train_y = train['lasso'] #what are we predicting 
test_y = train['lasso']

#train_x = train[['ch_5','ch_7','dc_5']]#,'ch_7','dc_4','dc_7']]
#test_x = test[['ch_5','ch_7','dc_5']]#,'ch_7','dc_4','dc_7']]

train_x = train[['ch_5','ch_7','dc_5']]#,'ch_7','dc_4','dc_7']]
test_x = test[['ch_5','ch_7','dc_5']]#,'ch_7','dc_4','dc_7']]

train_x_scaled  = preprocessing.normalize(train_x, norm='l1')
test_x_scaled  = preprocessing.normalize(test_x, norm='l1')

#train_x_scaled  = preprocessing.scale(train_x)
#test_x_scaled  = preprocessing.scale(test_x)


print (train_y.shape, train_x.shape) #y should by (datapoints,), x should by (datapoints,descriptors)

#break down unique batteris in test.train split 

(824,) (824, 3)


In [14]:
#Fit to a logistic Regressor 
######
#lin = linear_model.LogisticRegression()
#lin.fit(train_x_scaled,train_y)
#coef = lin.coef_
#c0,c1,c2,c3,c5,c6,d1,d2,d3,d4,d6
#coef

#trainpred=lin.predict(train_x) #predict train data 
#testpred=lin.predict(test_x) 

clf = NearestCentroid()
clf = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(train_x_scaled)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_x_scaled, train_y)
testpred=clf.predict(train_x_scaled)
trainpred=clf.predict(train_x_scaled)

In [15]:
nu_svc  = svm.NuSVC(kernel='linear').fit(train_x,train_y)
lin_svc = svm.LinearSVC().fit(train_x, train_y)

In [29]:
#Support Vector Machine 
#-->Chosen under the assumption that eventually more batteries will be added with highly dimensional features that 
# can not be seperated by simple linear or logistic regression models 

clf = svm.SVC(kernel='linear') #initiate SVM 
fit = clf.fit(train_x_scaled, train_y) #fit to training data
trainpred=clf.predict(train_x_scaled) #predict train data 
testpred=clf.predict(test_x_scaled)


In [30]:
trainpred=lin_svc.predict(train_x_scaled) #predict train data 
testpred=lin_svc.predict(test_x_scaled)

In [31]:
def accuracy(test_df,pred_df):
    """Calculates the amount of data classified correctly"""
    #Input for test_y should be the test dataframe (ex -- see below)
    N= len(pred_df)
    #print (N) 
    #print(test_df.names.size)
    #print(pred_df.size)
    compare = np.array([test_df.label, pred_df]).reshape(N,2)
    asses = pd.DataFrame (compare, columns = ['actual','predicted'])
    count = 0
    correct = 0
    for i in range(len(asses)):
        if asses.loc[i,('actual')] == asses.loc[i, ('predicted')]:
            correct += 1 
            count += 1 
            asses.loc[i, ('correct')] = 1
        else: 
            count += 1
            asses.loc[i, ('correct')] = 0
    return (1-correct/count)*100, asses

In [32]:
test_error, test_df,  = accuracy(test,testpred)
train_error, train_df,  = accuracy(train,trainpred)
print ("Testing error: ","%.2f" %test_error,"%")
print ("Training error: ","%.2f" %train_error,"%")

Testing error:  20.39 %
Training error:  24.03 %


In [ ]:
train_x
train_y = train['Label']

In [ ]:
pd_train_y = pd.DataFrame(train['label'])
frames = [train_x,pd_train_y]
datatot = pd.concat(frames, axis=1, ignore_index = True)
datatot.columns = 'ch_5', 'ch_7','dc_5', 'Label'
datatot.head()

In [ ]:
#fgframe = plt.figure(figsize=(5,5), facecolor='w', edgecolor='k')
#fg = sns.FacetGrid(data, hue='label', size = 15, palette = 'cool')
#fg.map(plt.scatter, 'dc_7','ch_4').add_legend()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl

In [ ]:
plt.rcParams.update({'font.size':12})

colors = ['red', 'blue']
datatot = datatot.reset_index(drop = True)
for i in range(len(datatot)):
    if datatot.loc[i, ('Label')] == 'LiCoO2':
        datatot.loc[i, ('Color')] = 'r'
    elif datatot.loc[i, ('Label')] == 'LiFePO4':
        datatot.loc[i, ('Color')] = 'b'
    else: 
        datatot.loc[i, ('Color')] = 'g'
#plt.scatter([datatot['ch_7'], datatot['dc_5'], datatot['ch_5'], c = datatot['Color'])
datatot.head()
             
fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot(111)#, projection='3d')
ax.scatter(datatot['ch_5'], datatot['ch_7'], c = datatot['Color'])
#ax.set_xlabel('Second Peak in Charge Cycle ($Ah/V$)') #ch7
ax.set_xlabel('First Peak in Charge Cycle ($Ah/V$)')
ax.set_ylabel('Second Peak in Charge Cycle ($Ah/V$)')

In [ ]:
datatot.head()
datatot = datatot.reset_index(drop = True)
datatot.head()

In [ ]:
pd_train_y = pd.DataFrame(train_y)
pd_train_y = pd_train_y.reset_index(drop = True)
#pd_train_y['label']

In [ ]:
for i in range(len(pd_train_y)):
    #print (i)
    if pd_train_y.loc[i,('label')] == 'LiCoO2':
        pd_train_y.loc[i,('num')] = 0
    else:
        pd_train_y.loc[i,('num')] = 1  

In [ ]:
#print(train_y.head())
print (pd_train_y.tail())
train_x[['ch_5','dc_5']].head()
#pd_train_y['num']

In [ ]:
# import some data to play with
#iris = datasets.load_iris()
X = train_x[['ch_5','dc_5']] # we only take the first two features. We could
                     # avoid this ugly slicing by using a two-dim dataset
Y = pd_train_y['num']

h=.02 # step size in the mesh

# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
#svc     = svm.SVC(kernel='linear').fit(X, Y)
#rbf_svc = svm.SVC(kernel='poly').fit(X, Y)
#nu_svc  = svm.NuSVC(kernel='linear').fit(X,Y)
#lin_svc = svm.LinearSVC().fit(X, Y)

# create a mesh to plot in
x_min, x_max = X.min()-1, X.max()+1
y_min, y_max = X.min()-1, X.max()+1

X.min()[0:3].shape

In [ ]:
# import some data to play with
#iris = datasets.load_iris

h=.02 # step size in the mesh

# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
#clf = svm.SVC(kernel='poly', degree = 4) #initiate SVM 
svm = clf.fit(train_x_scaled, train_y) #fit to training data
trainpred=clf.predict(train_x_scaled) #predict train data 
testpred=clf.predict(test_x_scaled)

# create a mesh to plot in
x_min, x_max = X.min()[0:1]-1, X.max()[0:1]+1
y_min, y_max = X.min()[1:2]-1, X.max()[1:2]+1
xx, yy = np.meshgrid(np.arange(x_min[0], x_max[0], h),np.arange(y_min[0], y_max[0], h))

# title for the plots
#titles = ['SVC with linear kernel',
         # 'SVC with polynomial (degree 3) kernel',
         # 'NuSVC with linear kernel',
         # 'LinearSVC (linear kernel)']


#pl.set_cmap(pl.cm.Paired)

#for i, clf in enumerate((svc, rbf_svc, nu_svc, lin_svc)):
    # Plot the decision boundary. For that, we will asign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
pl.set_cmap(pl.cm.Paired)
#pl.subplot(2, 2, i+1)
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
#trainpred=clf.predict(train_x_scaled) #predict train data 
#testpred=clf.predict(test_x_scaled)

    # Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.contourf(xx, yy, Z, cmap='coolwarm')
pl.axis('tight')

    # Plot also the training points
colors = ['red', 'blue']


for i in range(len(datatot)):
    if datatot.loc[i, ('Label')] == 'LiCoO2':
        datatot.loc[i, ('Color')] = 'r'
    elif datatot.loc[i, ('Label')] == 'LiFePO4':
        datatot.loc[i, ('Color')] = 'b'
    else: 
        datatot.loc[i, ('Color')] = 'g'
plt.scatter(train_x_scaled['ch_5'], train_x_scaled['dc_5'], c = datatot['Color'])
#plt.scatter(X['ch_5'], X['dc_5'], c='red')

#plt.title(titles[i])

plt.axis('tight')
plt.show()